In [1]:
import numpy as np
import pandas as pd
import pathlib
import random
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import layers, Sequential, regularizers

In [2]:
DATA_PATH = "../datasets/mushrooms.csv"   # update path as needed
KERAS_MODEL_PATH = "mushroom_model.keras" # saved Keras model (.keras)
TFLITE_MODEL_PATH = "mushroom_model_quantized.tflite"

In [3]:
df = pd.read_csv(DATA_PATH)

In [4]:
cols = df.columns.tolist()
assert 'class' in cols, "CSV must contain 'class' column"

In [5]:
X_df = df.drop(columns=['class']).copy()
y_ser = df['class'].copy()

In [6]:
X_enc = X_df.copy()
labelers = {}
for c in X_df.columns:
    le = LabelEncoder()
    X_enc[c] = le.fit_transform(X_df[c].astype(str))
    labelers[c] = le

In [7]:
le_y = LabelEncoder()
y = le_y.fit_transform(y_ser.astype(str))  # 0/1 array

In [8]:
X_np = X_enc.values.astype(np.float32)  # raw numeric (label-encoded) inputs
y_np = np.array(y).astype(np.int32)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_np, y_np, test_size=0.20, random_state=42, stratify=y_np
)

In [10]:
print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (6499, 22) (1625, 22) (6499,) (1625,)


In [11]:
def build_inverted_pyramid_with_normalization(input_dim, dropout_rates=None):
    if dropout_rates is None:
        dropout_rates = [0.3, 0.4, 0.4, 0.3, 0.3, 0.2, 0.2, 0.2]

    norm = layers.Normalization(axis=-1, dtype="float32")
    # We'll adapt norm on the training data below
    inp = layers.Input(shape=(input_dim,), dtype=tf.float32, name="input")
    x = norm(inp)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[0])(x)

    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[1])(x)

    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[2])(x)

    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[3])(x)

    x = layers.Dense(16, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[4])(x)

    x = layers.Dense(8, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[5])(x)

    x = layers.Dense(4, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[6])(x)

    x = layers.Dense(2, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[7])(x)

    out = layers.Dense(1, activation='sigmoid', dtype="float32")(x)
    model = tf.keras.Model(inputs=inp, outputs=out)
    # attach normalization layer instance so we can adapt it externally before training
    model.normalization_layer = norm
    return model

In [12]:
model = build_inverted_pyramid_with_normalization(input_dim=X_train.shape[1])
# adapt normalization on raw (label-encoded) training data
model.normalization_layer.adapt(X_train)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 22)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization (Normalization)   │ (None, 22)             │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4)              │            1

 Total params: 51,918 (202.81 KB)

 Trainable params: 50,853 (198.64 KB)

 Non-trainable params: 1,065 (4.16 KB)

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=1000,
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.5689 - loss: 0.7696 - val_accuracy: 0.5698 - val_loss: 0.6637
Epoch 2/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6563 - loss: 0.6184 - val_accuracy: 0.7631 - val_loss: 0.4961
Epoch 3/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7229 - loss: 0.5357 - val_accuracy: 0.9360 - val_loss: 0.2922
Epoch 4/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7895 - loss: 0.4583 - val_accuracy: 0.9625 - val_loss: 0.2091
Epoch 5/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8412 - loss: 0.3741 - val_accuracy: 0.9760 - val_loss: 0.1477
Epoch 6/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8614 - loss: 0.3339 - val_accuracy: 0.9877 - val_loss: 0.1099
Epoch 7/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8880 - loss: 0.2885 - val_accuracy: 0.9908 - val_loss: 0.0814
Epoch 8/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9100 - loss: 0.255

In [14]:
model.save(KERAS_MODEL_PATH)
print(f"Keras model saved to: {KERAS_MODEL_PATH}")

Keras model saved to: mushroom_model.keras


In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
y_pred_probs_keras = model.predict(X_test).flatten()
y_pred_keras = (y_pred_probs_keras > 0.5).astype(int)
print("\nKeras model accuracy:", accuracy_score(y_test, y_pred_keras))
print(classification_report(y_test, y_pred_keras, digits=4))

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

Keras model accuracy: 1.0
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       842
           1     1.0000    1.0000    1.0000       783

    accuracy                         1.0000      1625
   macro avg     1.0000    1.0000    1.0000      1625
weighted avg     1.0000    1.0000    1.0000      1625



In [18]:
def representative_dataset():
    for i in range(X_train.shape[0]):
        sample = X_train[i:i+1].astype(np.float32)
        yield [sample]

In [19]:
# Create converter and quantize with representative dataset (default optimizations)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
# NOTE: by not forcing target_spec, converter will choose quantization that fits model & repr. dataset.
tflite_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\pushp\AppData\Local\Temp\tmpmkmvt1le\assets


INFO:tensorflow:Assets written to: C:\Users\pushp\AppData\Local\Temp\tmpmkmvt1le\assets


Saved artifact at 'C:\Users\pushp\AppData\Local\Temp\tmpmkmvt1le'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 22), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2351568555088: TensorSpec(shape=(1, 22), dtype=tf.float32, name=None)
  2351568557008: TensorSpec(shape=(1, 22), dtype=tf.float32, name=None)
  2351567620816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567625616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567619664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567624272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567623120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567625232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567622736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567623504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2351567625040

e:\AgroML_Hub\models\.venv\Lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [20]:
with open(TFLITE_MODEL_PATH, "wb") as f:
    f.write(tflite_quant)
print(f"TFLite quantized model saved to: {TFLITE_MODEL_PATH}")

TFLite quantized model saved to: mushroom_model_quantized.tflite


In [21]:
def evaluate_tflite_model(tflite_file, X_raw, y_true):
    # X_raw: raw integer-encoded inputs (not normalized); model's Normalization layer is inside model
    interpreter = tf.lite.Interpreter(model_path=tflite_file)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    in_scale, in_zero_point = input_details[0].get('quantization', (0.0, 0))
    out_scale, out_zero_point = output_details[0].get('quantization', (0.0, 0))

    preds = []
    for i in range(X_raw.shape[0]):
        inp = X_raw[i:i+1].astype(np.float32)

        # If input is quantized, we must quantize the float input
        if in_scale and in_scale != 0:
            # quantize
            q_in = (inp / in_scale + in_zero_point).round().astype(input_details[0]['dtype'])
            interpreter.set_tensor(input_details[0]['index'], q_in)
        else:
            interpreter.set_tensor(input_details[0]['index'], inp.astype(input_details[0]['dtype']))

        interpreter.invoke()
        out = interpreter.get_tensor(output_details[0]['index'])
        # dequantize if output quantized
        if out_scale and out_scale != 0:
            out_deq = (out.astype(np.float32) - out_zero_point) * out_scale
        else:
            out_deq = out.astype(np.float32)
        prob = float(out_deq.flatten()[0])
        pred = 1 if prob > 0.5 else 0
        preds.append(pred)

    preds = np.array(preds, dtype=int)
    acc = accuracy_score(y_true, preds)
    print("\nTFLite model evaluation")
    print("Accuracy:", acc)
    print(classification_report(y_true, preds, digits=4))

In [22]:
evaluate_tflite_model(TFLITE_MODEL_PATH, X_test, y_test)


TFLite model evaluation
Accuracy: 0.5181538461538462
              precision    recall  f1-score   support

           0     0.5182    1.0000    0.6826       842
           1     0.0000    0.0000    0.0000       783

    accuracy                         0.5182      1625
   macro avg     0.2591    0.5000    0.3413      1625
weighted avg     0.2685    0.5182    0.3537      1625



e:\AgroML_Hub\models\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
e:\AgroML_Hub\models\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\AgroML_Hub\models\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [23]:
print("\nLabel encoder classes for 'class' (target):", list(le_y.classes_))
print("If you want to predict raw labels from model predictions, use le_y.inverse_transform([...])")


Label encoder classes for 'class' (target): ['e', 'p']
If you want to predict raw labels from model predictions, use le_y.inverse_transform([...])
